# Dataset auto-labelling

Goals:
* label and exclude some hard-to-learn sounds
* other?

---

# Harmonic / Percussive / Other labels

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
#mpl.rcParams['figure.dpi'] = 300  # No widget
%matplotlib widget
import librosa
import librosa.display
import librosa.decompose
import librosa.core
import importlib
import warnings
from IPython.display import Audio

import config
import data.dataset

# fft parameters - unused? It is much easier to use librosa's defaults...
n_fft = 1024
fft_hop = 256
Fs = 22050

# Dataset spectrogram won't be used here
dataset = data.dataset.DexedDataset(** data.dataset.model_config_to_dataset_kwargs(config.model))
print(dataset)
preset_UIDs = dataset.valid_preset_UIDs

Dataset of 30293/30293 Dexed presets. 144 learnable synth params, 11 fixed params. Mel Spectrogram items, size=torch.Size([1, 257, 347]), min=-120.0dB, normalization:min_max


## Critères de séparation
Pour les sons de synthé bien harmoniques, aucun problème, le spectre de puissance harmonique (margin 3.0) contient bien >98% de la puissance spectrale totale.
Attention par contre, on peut descendre à - de 50% d'énergie harmonique pour des synthés à forte attaque (type percu). Mais on a alors pas beaucoup de résidus.

Pour des sons identifiés comme harmoniques "qui partent en sucette", ça va être dûr.

Pour des sons SFX avec des composantes harmonique aléatoires : 
* **TODO voir si les raies principales correspondent à la note jouée** (pitch tracking)

Pour des sons SFX n'imp :
* Spectral flatness?
* Pas d'harmoniques faciles à détecter?
* Pas de pitch clair?

* Sons harmoniques mais qui partent en sucette :
    * 3104 (64% res), 3011
    * 5760
    * 7909
    * 31421 (65% harm mais pitch descend très bas)
    * 71706 (quasi-séquence) (56% perc)
    * 75897 (vibrato 12%;29% compté en perc;res)
* Sons harmoniques qui partent en sucette au release :
    * 5833 (22% de résidu, quasi-tout au release)
* SFX, sons intéressants mais pas harmoniques (pourraient aider le NN à bien comprendre comment fonctionne Dexed ?)
    * 2520, 2522 (68% perc)
    * 3067
    * 3103, 3105, 3107, 3109, 3110, 3112, 3115, 3116, 3118, ... (pas mal de résidus, perc pour certains)
    * 4200 (rés 62%)
    * 7059
    * 7893
    * 8123
    * 22468
    * 54099
    * 71949
    * 73029 (36/24/40 %)
    * 75989
    * 76103, 76105
    * 78064
    * 203867
    * 245768
* Kicks / percus (idem, peut-être intéressants)
    * 81538 (77% perc)
    * 246879 (95% prec)
* Presque zéro :
    * 2555 (96% harmonique mais à 20Hz)
    * 54035 (100% perc)
* Gros n'imp, limite inécoutable :
    * 3111 (10% harm)
    * 71429 (29% harm)
    * 74146 (12/17/71 %)
    * 75334 (11/11/78 %), 75340 (10/20/70%)
    * 78256 (40/7/54 % mais harmoniques dégueu)
    * 78707 (10/31/59 %)
    * 81660 (12/24/64 %), 81662 (9/16/75 %, sinus dégueu pas compté en harmonique), 81664 (9/20/71 %), ...
    * 171019 (22/74/4 %, sinus sur-aigu pas compté en harmonique)

In [2]:
# - - - - - - - - - - - - - - - - - - - - - - - -
preset_index = 300
preset_UID = preset_UIDs[preset_index]
preset_UID =  81664 # possible d'entrer direct un UID ici - commenter sinon
# - - - - - - - - - - - - - - - - - - - - - - - -
# Test on 1 sample: harmonic/percussive source separation
x_wav, Fs = dataset._get_wav_file(preset_UID)

import utils.audio
importlib.reload(utils.audio)
sample_labeler = utils.audio.SimpleSampleLabeler(x_wav, Fs, hpss_margin=3.0)
sample_labeler.plot_hpr_specs()

sample_labeler.print_labels()

print("attack energies = {}".format(sample_labeler.attack_energies))

print("Original audio, preset UID={}".format(preset_UID))
Audio(x_wav, rate=Fs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

is_harmonic=False   is_percussive=False
attack energies = {'D': 4669.377727601873, 'H': 516.317720714559, 'P': 3565.369320136998, 'R': 587.6906867503164}
Original audio, preset UID=81664


/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'linscaley' parameter of __init__() has been renamed 'linscale' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)


In [3]:
print("Harmonic content")
Audio(sample_labeler.get_harmonic_sound(), rate=Fs)

Harmonic content


In [4]:
print("Percussive content")
Audio(sample_labeler.get_percussive_sound(), rate=Fs)

Percussive content


In [5]:
print("Residual content")
Audio(sample_labeler.get_residual_sound(), rate=Fs)

Residual content


---
## Exemples d'auto-labelling

In [11]:
importlib.reload(utils.audio)

found = False
while not found:
    # - - - - - - - - - - - - - - - - - - - - - - - -
    preset_index = np.random.randint(0, len(dataset))
    preset_UID = preset_UIDs[preset_index]
    #preset_UID = 28  # possible d'entrer direct un UID ici - commenter sinon
    # - - - - - - - - - - - - - - - - - - - - - - - -
    # Test on 1 sample: harmonic/percussive source separation
    x_wav, Fs = dataset._get_wav_file(preset_UID)
    sample_labeler = utils.audio.SimpleSampleLabeler(x_wav, Fs, hpss_margin=3.0)
    
    #found = sample_labeler.is_percussive()
    found = sample_labeler.is_percussive and not sample_labeler.is_harmonic


print("Original audio, preset UID={}".format(preset_UID))
sample_labeler.print_labels()
sample_labeler.plot_hpr_specs()

Audio(x_wav, rate=Fs)

Original audio, preset UID=76014
is_harmonic=False   is_percussive=True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/home/gwendal/anaconda3/lib/python3.8/site-packages/librosa/display.py:974: MatplotlibDeprecationWarning: The 'linscaley' parameter of __init__() has been renamed 'linscale' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)


In [8]:
# Stats sur le dataset complet
importlib.reload(utils.audio)
nb = {'harmonic': 0, 'percussive': 0, 'sfx': 0}
if False:
    for preset_index in range(len(dataset)):
        x_wav, Fs = dataset._get_wav_file(preset_UIDs[preset_index])
        sample_labeler = utils.audio.SimpleSampleLabeler(x_wav, Fs, hpss_margin=3.0)
        for k in nb:
            nb[k] += 1 if sample_labeler.has_label(k) else 0
        if preset_index % 1000 == 0:
            print(nb)
            if preset_index > 0:
                break
nb

{'harmonic': 0, 'percussive': 0, 'sfx': 0}

In [3]:
# =========================================================
# =========================================================
# ========== Add labels to Dexed SQLite database ==========
# =========================================================
# =========================================================
# !!! DANGER ZONE !!!
import sqlite3
conn = sqlite3.connect('./synth/dexed_presets.sqlite', detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()
# column 'labels' has already been inserted (from sqlite db browser)
# TODO add labels are comma-separated text values
for preset_UID in dataset.valid_preset_UIDs:
    print(preset_UID)
    if preset_UID > 20:  # TEMP, pour tests
        break

conn.commit()
conn.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


---
## Pitch analysis
todo...

In [12]:
# Pitch?
pitches, magnitudes = librosa.core.piptrack(S=sample_labeler.specs['H'], fmin=150, fmax=4e3)

# TODOOOO les pitches! sur les 3 secondes où la note joue

# On peut prendre les 100 + basses fréquences (ça va déjà jusqu'à quasi 1kHz)
plt.figure(figsize=(8, 6))  
plt.subplot(2, 1, 1)
plt.imshow(pitches[:100, :], aspect="auto", interpolation="nearest", origin='lower')
plt.subplot(2, 1, 2)
plt.imshow(magnitudes[:100, :], aspect="auto", interpolation="nearest", origin='lower')
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …